In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import pickle

In [3]:
df = pd.read_csv("Amazon Sale Report.csv",low_memory=False)

In [4]:
df

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,128970,406-6001380-7673107,05-31-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,...,INR,517.00,HYDERABAD,TELANGANA,500013.0,IN,NaN,False,NaN,False
128971,128971,402-9551604-7544318,05-31-22,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,...,INR,999.00,GURUGRAM,HARYANA,122004.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False
128972,128972,407-9547469-3152358,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,...,INR,690.00,HYDERABAD,TELANGANA,500049.0,IN,NaN,False,NaN,False
128973,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,INR,1199.00,Halol,Gujarat,389350.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

# Removing index and order id

In [6]:
df.drop("index",axis=1,inplace=True)

In [7]:
df.drop("Order ID",axis=1,inplace=True)

In [8]:
df["Date"].unique()

array(['04-30-22', '04-29-22', '04-28-22', '04-27-22', '04-26-22',
       '04-25-22', '04-24-22', '04-23-22', '04-22-22', '04-21-22',
       '04-20-22', '04-19-22', '04-18-22', '04-17-22', '04-16-22',
       '04-15-22', '04-14-22', '04-13-22', '04-12-22', '04-11-22',
       '04-10-22', '04-09-22', '04-08-22', '04-07-22', '04-06-22',
       '04-05-22', '04-04-22', '04-03-22', '04-02-22', '04-01-22',
       '03-31-22', '05-31-22', '05-30-22', '05-29-22', '05-28-22',
       '05-27-22', '05-26-22', '05-25-22', '05-24-22', '05-23-22',
       '05-22-22', '05-21-22', '05-20-22', '05-19-22', '05-18-22',
       '05-17-22', '05-16-22', '05-15-22', '05-14-22', '05-13-22',
       '05-12-22', '05-11-22', '05-10-22', '05-09-22', '05-08-22',
       '05-07-22', '05-06-22', '05-05-22', '05-04-22', '05-03-22',
       '05-02-22', '05-01-22', '06-29-22', '06-28-22', '06-27-22',
       '06-26-22', '06-25-22', '06-24-22', '06-23-22', '06-22-22',
       '06-21-22', '06-20-22', '06-19-22', '06-18-22', '06-17-

In [9]:
df["Status"].unique()

array(['Cancelled', 'Shipped - Delivered to Buyer', 'Shipped',
       'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer',
       'Shipped - Lost in Transit', 'Shipped - Out for Delivery',
       'Shipped - Returning to Seller', 'Shipped - Picked Up', 'Pending',
       'Pending - Waiting for Pick Up', 'Shipped - Damaged', 'Shipping'],
      dtype=object)

In [10]:
df["Status"].nunique()

13

In [11]:
def col_val_replace(df,col):
  repl_dict={}
  df[col]=df[col].map(lambda x: x.upper() if type(x)==str else x)
  for i in df[col].unique():
    print("Enter replacement for ", i)
    ip = input()
    if len(ip)==0:
      repl_dict[i]=i
    else:
      repl_dict[i]=ip
  df[col] = df[col].map(lambda x: repl_dict[x])

In [12]:
col_val_replace(df,"Status")

Enter replacement for  CANCELLED

Enter replacement for  SHIPPED - DELIVERED TO BUYER
SHIPPED
Enter replacement for  SHIPPED

Enter replacement for  SHIPPED - RETURNED TO SELLER
SHIPPED
Enter replacement for  SHIPPED - REJECTED BY BUYER
SHIPPED
Enter replacement for  SHIPPED - LOST IN TRANSIT
SHIPPED
Enter replacement for  SHIPPED - OUT FOR DELIVERY
SHIPPED
Enter replacement for  SHIPPED - RETURNING TO SELLER
SHIPPED
Enter replacement for  SHIPPED - PICKED UP
SHIPPED
Enter replacement for  PENDING
SHIPPED
Enter replacement for  PENDING - WAITING FOR PICK UP
SHIPPED
Enter replacement for  SHIPPED - DAMAGED
SHIPPED
Enter replacement for  SHIPPING
SHIPPED


In [13]:
df["Status"].nunique()

2

In [14]:
df["Fulfilment"].unique()

array(['Merchant', 'Amazon'], dtype=object)

In [16]:
df.rename(columns={"Sales Channel ":"Sales Channel"},inplace = True)

In [17]:
df["Sales Channel"].unique()

array(['Amazon.in', 'Non-Amazon'], dtype=object)

In [18]:
df["ship-service-level"].unique()

array(['Standard', 'Expedited'], dtype=object)

In [19]:
df["Style"].nunique()

1377

In [20]:
df["SKU"].nunique()

7195

In [21]:
df["Category"].nunique()

9

In [22]:
df["Size"].unique()

array(['S', '3XL', 'XL', 'L', 'XXL', 'XS', '6XL', 'M', '4XL', '5XL',
       'Free'], dtype=object)

In [23]:
df["ASIN"].nunique()

7190

In [24]:
df["Courier Status"].unique()

array([nan, 'Shipped', 'Cancelled', 'Unshipped'], dtype=object)

In [25]:
df.drop("Courier Status",axis=1,inplace=True)

In [26]:
df["Qty"].unique()

array([ 0,  1,  2, 15,  3,  9, 13,  5,  4,  8], dtype=int64)

# Dropping currency column as having only one unique value 

In [27]:
df["currency"].unique()

array(['INR', nan], dtype=object)

In [28]:
df.drop("currency",axis=1,inplace=True)

In [30]:
df["ship-city"].nunique()

8955

In [31]:
df["ship-city"] = df["ship-city"].map(lambda x: x.upper() if type(x)==str else x)

In [32]:
df["ship-city"].nunique()

7297

In [33]:
df["ship-state"].unique() 

array(['MAHARASHTRA', 'KARNATAKA', 'PUDUCHERRY', 'TAMIL NADU',
       'UTTAR PRADESH', 'CHANDIGARH', 'TELANGANA', 'ANDHRA PRADESH',
       'RAJASTHAN', 'DELHI', 'HARYANA', 'ASSAM', 'JHARKHAND',
       'CHHATTISGARH', 'ODISHA', 'KERALA', 'MADHYA PRADESH',
       'WEST BENGAL', 'NAGALAND', 'Gujarat', 'UTTARAKHAND', 'BIHAR',
       'JAMMU & KASHMIR', 'PUNJAB', 'HIMACHAL PRADESH',
       'ARUNACHAL PRADESH', 'MANIPUR', 'Goa', 'MEGHALAYA', 'GOA',
       'TRIPURA', 'LADAKH', 'DADRA AND NAGAR', 'SIKKIM', 'Delhi', nan,
       'ANDAMAN & NICOBAR ', 'Punjab', 'Rajshthan', 'Manipur',
       'rajasthan', 'Odisha', 'NL', 'Bihar', 'MIZORAM', 'punjab',
       'New Delhi', 'Rajasthan', 'Punjab/Mohali/Zirakpur', 'Puducherry',
       'delhi', 'RJ', 'Chandigarh', 'orissa', 'LAKSHADWEEP', 'goa', 'PB',
       'APO', 'Arunachal Pradesh', 'AR', 'Pondicherry', 'Sikkim',
       'Arunachal pradesh', 'Nagaland', 'bihar', 'Mizoram', 'rajsthan',
       'Orissa', 'Rajsthan', 'Meghalaya'], dtype=object)

In [34]:
df["ship-state"].nunique()

69

In [35]:
col_val_replace(df,"ship-state")

Enter replacement for  MAHARASHTRA

Enter replacement for  KARNATAKA

Enter replacement for  PUDUCHERRY

Enter replacement for  TAMIL NADU

Enter replacement for  UTTAR PRADESH

Enter replacement for  CHANDIGARH

Enter replacement for  TELANGANA

Enter replacement for  ANDHRA PRADESH

Enter replacement for  RAJASTHAN

Enter replacement for  DELHI

Enter replacement for  HARYANA

Enter replacement for  ASSAM

Enter replacement for  JHARKHAND

Enter replacement for  CHHATTISGARH

Enter replacement for  ODISHA

Enter replacement for  KERALA

Enter replacement for  MADHYA PRADESH

Enter replacement for  WEST BENGAL

Enter replacement for  NAGALAND

Enter replacement for  GUJARAT

Enter replacement for  UTTARAKHAND

Enter replacement for  BIHAR

Enter replacement for  JAMMU & KASHMIR

Enter replacement for  PUNJAB

Enter replacement for  HIMACHAL PRADESH

Enter replacement for  ARUNACHAL PRADESH

Enter replacement for  MANIPUR

Enter replacement for  GOA

Enter replacement for  MEGHALAYA

E

In [36]:
df["ship-state"].nunique()

37

In [37]:
df["ship-state"].unique()

array(['MAHARASHTRA', 'KARNATAKA', 'PUDUCHERRY', 'TAMIL NADU',
       'UTTAR PRADESH', 'CHANDIGARH', 'TELANGANA', 'ANDHRA PRADESH',
       'RAJASTHAN', 'DELHI', 'HARYANA', 'ASSAM', 'JHARKHAND',
       'CHHATTISGARH', 'ODISHA', 'KERALA', 'MADHYA PRADESH',
       'WEST BENGAL', 'NAGALAND', 'GUJARAT', 'UTTARAKHAND', 'BIHAR',
       'JAMMU & KASHMIR', 'PUNJAB', 'HIMACHAL PRADESH',
       'ARUNACHAL PRADESH', 'MANIPUR', 'GOA', 'MEGHALAYA', 'TRIPURA',
       'LADAKH', 'DADRA AND NAGAR', 'SIKKIM', nan, 'ANDAMAN & NICOBAR ',
       'MIZORAM', 'LAKSHADWEEP', 'APO'], dtype=object)

In [38]:
df.to_csv("Amazon processed.csv",index=False)

In [39]:
df["ship-postal-code"].nunique()

9459

In [40]:
df["ship-city"].nunique()

7297

In [41]:
df.loc[:,["ship-city","ship-postal-code"]].value_counts()

ship-city                ship-postal-code
NOIDA                    201301.0            972
GURUGRAM                 122001.0            572
BENGALURU                560068.0            570
                         560037.0            556
                         560076.0            512
                                            ... 
KOCH BIHAR               736171.0              1
KIZHUVALAM KOONTHALLOOR  695304.0              1
KIZHUPARAMBA             673645.0              1
KISHANGARH RENWAL        303603.0              1
KOTANANDURU              533407.0              1
Length: 14406, dtype: int64

In [42]:
df.drop("ship-city",axis=1,inplace=True)

In [43]:
df["ship-country"].unique()

array(['IN', nan], dtype=object)

In [44]:
df.drop("ship-country",axis=1,inplace=True)

In [45]:
df["promotion-ids"].nunique()

5787

In [46]:
df["B2B"].nunique()

2

In [47]:
df["fulfilled-by"].nunique()

1

In [48]:
df.drop("fulfilled-by",axis=1,inplace=True)

In [49]:
df["Unnamed: 22"].nunique()

1

In [50]:
df.drop("Unnamed: 22",axis=1,inplace=True)

In [51]:
df.isna().sum()

Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Qty                       0
Amount                 7795
ship-state               33
ship-postal-code         33
promotion-ids         49153
B2B                       0
dtype: int64

In [52]:
df.isna().sum()/df.shape[0]*100

Date                   0.000000
Status                 0.000000
Fulfilment             0.000000
Sales Channel          0.000000
ship-service-level     0.000000
Style                  0.000000
SKU                    0.000000
Category               0.000000
Size                   0.000000
ASIN                   0.000000
Qty                    0.000000
Amount                 6.043807
ship-state             0.025586
ship-postal-code       0.025586
promotion-ids         38.110487
B2B                    0.000000
dtype: float64

In [53]:
df["Status"].value_counts()

SHIPPED      110643
CANCELLED     18332
Name: Status, dtype: int64

In [54]:
df["ship-state"].isnull().sum()

33

In [55]:
df.shape

(128975, 16)

In [56]:
df = df.loc[df["ship-state"].isnull()==False]

In [57]:
df.shape

(128942, 16)

In [58]:
df.isna().sum()/df.shape[0]*100

Date                   0.000000
Status                 0.000000
Fulfilment             0.000000
Sales Channel          0.000000
ship-service-level     0.000000
Style                  0.000000
SKU                    0.000000
Category               0.000000
Size                   0.000000
ASIN                   0.000000
Qty                    0.000000
Amount                 6.043803
ship-state             0.000000
ship-postal-code       0.000000
promotion-ids         38.114036
B2B                    0.000000
dtype: float64

In [59]:
df["promotion-ids"] = df["promotion-ids"].map(lambda x: x.split(" ") if type(x) == str else x )\
                                         .map(lambda x: x[:3] if type(x) == list else x)\
                                         .map(lambda x: " ".join(x) if type(x) == list else x)

C:\Users\PARESH~1\AppData\Local\Temp/ipykernel_23164/3415882297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["promotion-ids"] = df["promotion-ids"].map(lambda x: x.split(" ") if type(x) == str else x )\


In [60]:
df["promotion-ids"].nunique()

14

In [ ]:
df["promotion-ids"].isna().sum()

In [ ]:
df["promotion-ids"].fillna("no promo",inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df["Amount"].mean()

In [ ]:
df["Amount"].median()

In [ ]:
plt.hist(df["Amount"],bins=100)

In [ ]:
df["Amount"].fillna(df["Amount"].median(),inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.to_csv("Amazon processed2.csv",index=False)

In [ ]:
df.shape

In [ ]:
df = df.loc[df["ship-state"] != "APO",]

In [ ]:
df.shape

In [ ]:
df["SKU"].unique()

In [ ]:
df["SKU"].nunique()

In [ ]:
df["SKU"] = df["SKU"].map(lambda x: x.split("-") if type(x) == str else x )\
                     .map(lambda x: x[:2] if type(x) == list else x)\
                     .map(lambda x: "-".join(x) if type(x) == list else x)

In [ ]:
df["SKU"].nunique()

In [ ]:
df["Style"].unique()

In [ ]:
df["Style"].nunique()

In [ ]:
df[["Style","SKU"]].value_counts()

In [ ]:
df.drop("SKU",axis=1,inplace=True)

In [ ]:
df.to_csv("Amazon processed3.csv",index=False)

In [ ]:
df.drop("Date",axis=1,inplace=True)

In [ ]:
df3 = pd.get_dummies(df,drop_first=True)

In [ ]:
sel = SelectFromModel(DecisionTreeClassifier(random_state=7 ),
                      threshold = "mean")

In [ ]:
X = df3.drop("Status_SHIPPED", axis=1)
y = df3["Status_SHIPPED"]

In [ ]:
sel.fit(X,y)

In [ ]:
sel.estimator_.feature_importances_

In [ ]:
X_DT = sel.transform(X)

In [ ]:
X_DT.shape

In [ ]:
X.loc[:,sel.estimator_.feature_importances_>sel.estimator_.feature_importances_.mean()].columns

In [ ]:
cbc = CatBoostClassifier(random_state=7)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_DT,y,test_size=0.3,
                                                 random_state=7,
                                                 stratify=y)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
cbc.fit(X_train,y_train)

In [ ]:
y_pred = cbc.predict(X_test)

In [ ]:
f1_score(y_test,y_pred, average="micro")

In [ ]:
precision_score(y_test,y_pred)

In [ ]:
recall_score(y_test,y_pred)

In [ ]:
sel = SelectFromModel(CatBoostClassifier(random_seed=7),
                      threshold = "mean")

In [ ]:
sel.fit(X_train,y_train)

In [ ]:
X.loc[:,sel.estimator_.feature_importances_>sel.estimator_.feature_importances_.mean()].columns

In [ ]:
X_train_cbc = sel.transform(X_train)
X_test_cbc = sel.transform(X_test)

In [ ]:
X_train_cbc.shape, X_test_cbc.shape

In [ ]:
cbc = CatBoostClassifier(random_state=7)

In [ ]:
cbc.fit(X_train_cbc,y_train)

In [ ]:
y_pred = cbc.predict(X_test_cbc)

In [ ]:
precision_score(y_test,y_pred), recall_score(y_test,y_pred), f1_score(y_test,y_pred, average="micro")

In [ ]:
df1 = df.drop("Qty",axis=1)

In [ ]:
df1_ohe = pd.get_dummies(df1,drop_first=True)

In [ ]:
X1 = df1_ohe.drop("Status_SHIPPED",axis=1)
y1 = df1_ohe["Status_SHIPPED"]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X1,y1,test_size=0.3,
                                                 random_state=7,
                                                 stratify=y)

In [ ]:
sel = SelectFromModel(CatBoostClassifier(random_seed=7,iterations=500),
                      threshold = "mean")

In [ ]:
sel.fit(X_train,y_train)

In [ ]:
X1.loc[:,sel.estimator_.feature_importances_>sel.estimator_.feature_importances_.mean()].columns

In [ ]:
X_train_cbc = sel.transform(X_train)
X_test_cbc = sel.transform(X_test)

In [ ]:
X_train_cbc.shape, X_test_cbc.shape

In [ ]:
cbc = CatBoostClassifier(random_state=7,iterations=500)

In [ ]:
cbc.fit(X_train_cbc,y_train)

In [ ]:
y_pred = cbc.predict(X_test_cbc)

In [ ]:
precision_score(y_test,y_pred),recall_score(y_test,y_pred),f1_score(y_test,y_pred, average="micro")

In [ ]:
df_asin_drop = df1.drop("ASIN",axis=1)

In [ ]:
df_asin_drop["ship-postal-code"] = df_asin_drop["ship-postal-code"].astype("object")

In [ ]:
df_posco_ohe = pd.get_dummies(df_asin_drop,drop_first=True)

In [ ]:
X_pc = df_posco_ohe.drop("Status_SHIPPED",axis=1)
y_pc = df_posco_ohe["Status_SHIPPED"]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_pc,y_pc,test_size=0.3,
                                                 random_state=7,
                                                 stratify=y)

In [ ]:
sel = SelectFromModel(CatBoostClassifier(random_seed=7,iterations=500),
                      threshold = "mean")

In [ ]:
sel.fit(X_train,y_train)

In [ ]:
X_pc.loc[:,sel.estimator_.feature_importances_>sel.estimator_.feature_importances_.mean()].columns

In [ ]:
X_train_cbc = sel.transform(X_train)
X_test_cbc = sel.transform(X_test)

In [ ]:
X_train_cbc.shape, X_test_cbc.shape

In [ ]:
cbc = CatBoostClassifier(random_state=7,iterations=500)

In [ ]:
cbc.fit(X_train_cbc,y_train)

In [ ]:
y_pred = cbc.predict(X_test_cbc)

In [ ]:
precision_score(y_test,y_pred),recall_score(y_test,y_pred),f1_score(y_test,y_pred, average="micro")

In [ ]:
param_grid = {"n_estimators":[100,350],
             "min_samples_split":[0.01,0.05],
             "max_depth":[20,30]}

In [ ]:
gcv = GridSearchCV(estimator = RandomForestClassifier(bootstrap=True,oob_score=True,random_state = 7),param_grid=param_grid)

In [ ]:
gcv.fit(X_train_cbc,y_train)

In [ ]:
gcv.best_score_

In [ ]:
gcv.best_params_

In [ ]:
param_grid2 = {"n_estimators":[350,500],
             "min_samples_split":[0.01,0.005],
             "max_depth":[20,10]}

In [ ]:
gcv2 = GridSearchCV(estimator = RandomForestClassifier(bootstrap=True,oob_score=True,random_state = 7),param_grid=param_grid2,verbose=2)

In [ ]:
gcv2.fit(X_train_cbc,y_train)

In [ ]:
gcv2.best_score_

In [ ]:
gcv2.best_params_

In [ ]:
rf = RandomForestClassifier(bootstrap=True,oob_score=True,random_state = 7,max_depth=20,min_samples_split=0.005,n_estimators=500)

In [ ]:
rf.fit(X_train_cbc,y_train)

In [ ]:
y_pred_rf = rf.predict(X_test_cbc)

In [ ]:
precision_score(y_test,y_pred_rf),recall_score(y_test,y_pred_rf),f1_score(y_test,y_pred_rf, average="micro")

In [ ]:
param_grid_cbc = {"depth":[6,10,20],
                 "l2_leaf_reg":[0.1,1,10],
                 "border_count":[128,254]}

In [ ]:
gcv_cbc = GridSearchCV(estimator = CatBoostClassifier(random_seed=7,iterations=500)\
                       ,param_grid=param_grid_cbc,cv=2,verbose=2)

In [ ]:
gcv_cbc.fit(X_train_cbc,y_train)

In [ ]:
gcv_cbc.best_score_

In [ ]:
gcv_cbc.best_params_

In [ ]:
cbc_gcv_model = CatBoostClassifier(random_state=7,iterations=500,border_count=254,depth=6,l2_leaf_reg=1)

In [ ]:
cbc_gcv_model.fit(X_train_cbc,y_train)

In [ ]:
y_cbc_gcv_pred = cbc_gcv_model.predict(X_test_cbc)

In [ ]:
precision_score(y_test,y_cbc_gcv_pred),recall_score(y_test,y_cbc_gcv_pred),f1_score(y_test,y_cbc_gcv_pred, average="micro")